# Imports

In [11]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.nn.utils.rnn import pad_sequence

import pandas as pd
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# Tokenizer

In [12]:
class Tokenizer:
    def __init__(self, time_count: int = 260, note_count: int = 110, vel_count: int = 2):
        self.val_to_velo_id: dict = {i: i + 1 for i in range(vel_count)}
        self.val_to_note_id: dict = {i: i + 1 + vel_count for i in range(note_count)}
        self.val_to_time_id: dict = {i: i + 1 + vel_count + note_count for i in range(time_count)}

        self.velo_id_to_val: dict = {v: k for k, v in self.val_to_velo_id.items()}
        self.note_id_to_val: dict = {v: k for k, v in self.val_to_note_id.items()}
        self.time_id_to_val: dict = {v: k for k, v in self.val_to_time_id.items()}
        
        self.id_to_token: dict = {
            **{self.val_to_velo_id[i]: f'velo_{i}' for i in self.val_to_velo_id},
            **{self.val_to_note_id[i]: f'note_{i}' for i in self.val_to_note_id},
            **{self.val_to_time_id[i]: f'time_{i}' for i in self.val_to_time_id},
            0: '<pad>',
            vel_count + note_count + time_count + 1: '<bos>',
            vel_count + note_count + time_count + 2: '<eos>'
        }
        
        self.token_to_id: dict = {v: k for k, v in self.id_to_token.items()}
    

    def tuple_to_ids(self, tuple: tuple):
        return [self.val_to_time_id[tuple[0]], self.val_to_note_id[tuple[1]], self.val_to_velo_id[tuple[2]]]
    

    def tuple_list_to_ids(self, tuple_list: list[tuple]):
        l = []
        for t in tuple_list:
            l.extend(self.tuple_to_ids(t))
        return l


    def id_list_to_tuple_list(self, id_list: list[int]):
        l = []
        for i in range(0, len(id_list), 3):
            if i + 3 > len(id_list):
                break
            t = []
            for j, d in enumerate([self.time_id_to_val, self.note_id_to_val, self.velo_id_to_val]):
                if min(d) <= id_list[i+j] <= max(d):
                    t.append(d[id_list[i+j]])
                else:
                    t.append(-1)
            l.append(tuple(t))
        return l

# Dataset

In [13]:
class FrameDataset(Dataset):
    def __init__(self, image_midi_path_pairs: list[tuple], tokenizer: Tokenizer, transform=None, max_len=600):
        self.df = image_midi_path_pairs
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.transform = transform if transform else transforms.Compose([
            transforms.ToTensor()
        ])

    
    def __len__(self):
        return len(self.df)


    def __getitem__(self, idx):
        image_path = self.df.iloc[idx]['image']
        midi_path = self.df.iloc[idx]['midi']
        
        image = Image.open(image_path).convert('L')
        image = self.transform(image)

        midi = pd.read_csv(midi_path)
        midi['time'] = midi['time'] // 10
        midi['velocity'] = (midi['velocity'] > 0).astype(int)
        midi = midi.values.tolist()
        midi = self.tokenizer.tuple_list_to_ids(midi)
        midi.insert(0, self.tokenizer.token_to_id['<bos>'])
        midi.append(self.tokenizer.token_to_id['<eos>'])
        midi.extend([self.tokenizer.token_to_id['<pad>']] * (self.max_len - len(midi)))
        midi = torch.tensor(midi, dtype=torch.long)
        
        return image, midi

# Model

In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 600):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)
    
    def forward(self, x: torch.Tensor):
        x = x + self.encoding[:, :x.size(1), :]
        return x

In [15]:
class PianoTranscriber(nn.Module):
    def __init__(self, input_size: int, vocab_size: int, d_model: int = 128, nhead: int = 2, num_layers: int = 2):
        super(PianoTranscriber, self).__init__()

        self.input_layer = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)

        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers)

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.output = nn.Linear(d_model, vocab_size)

    def forward(self, src: torch.Tensor, tgt: torch.Tensor):
        # print(f'input: {src.shape}')
        src = src.flatten(1, 2)
        # print(f'flattened: {src.shape}')
        src = src.permute(0, 2, 1)
        # print(f'permuted: {src.shape}')
        src = self.input_layer(src)
        # print(f'input layer: {src.shape}')
        src = self.pos_encoder(src)
        # print(f'positional encoding: {src.shape}')
        src = src.permute(1, 0, 2)
        # print(f'permuted: {src.shape}')
        memory = self.encoder(src)
        # print(f'encoder: {memory.shape}')

        # print()
        # print(f'target: {tgt.shape}')
        tgt = self.embedding(tgt)
        # print(f'embedding: {tgt.shape}')
        tgt = self.pos_encoder(tgt)
        # print(f'positional encoding: {tgt.shape}')
        tgt = tgt.permute(1, 0, 2)
        # print(f'permuted: {tgt.shape}')

        tgt_mask = self.generate_square_subsequent_mask(tgt.size(0)).to(tgt.device)

        output = self.decoder(tgt, memory, tgt_mask=tgt_mask)
        # print(f'decoder: {output.shape}')
        output = self.output(output)
        # print(f'output: {output.shape}')
        output = output.permute(1, 0, 2)
        # print(f'permuted: {output.shape}')
        # print()
        return output

    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)

# TMP

In [16]:
df = pd.read_csv('dataset.csv')
df['image'] = df['image'].apply(lambda x: os.path.join('dataset', x))
df['midi'] = df['midi'].apply(lambda x: os.path.join('dataset', x))
df

,image,midi
0,dataset/252_0.png,dataset/252_0.csv
1,dataset/252_1.png,dataset/252_1.csv
2,dataset/252_2.png,dataset/252_2.csv
3,dataset/252_3.png,dataset/252_3.csv
4,dataset/252_4.png,dataset/252_4.csv
...,...,...
507,dataset/556_57.png,dataset/556_57.csv
508,dataset/556_58.png,dataset/556_58.csv
509,dataset/556_59.png,dataset/556_59.csv
510,dataset/556_60.png,dataset/556_60.csv


In [17]:
transform = transforms.Compose([
    transforms.Resize((128, 64)),
    transforms.ToTensor(),
])
tokenizer = Tokenizer()
dataset = FrameDataset(df, tokenizer, transform=transform, max_len=600)
loader = DataLoader(dataset, batch_size=2)
for i, (image, midi) in enumerate(loader):
    # plt.imshow(image[0].permute(1, 2, 0), origin='lower')
    if i == 0:
        break

In [18]:
model = PianoTranscriber(128, len(tokenizer.id_to_token)).to(device)

for i, (image, midi) in enumerate(loader):
    image = image.to(device)
    midi = midi.to(device)
    output = model(image, midi)
    output = output.argmax(dim=2)
    output = output[0]
    output = output.cpu().numpy().tolist()
    print(len(output))
    print(output)
    output = tokenizer.id_list_to_tuple_list(output)
    print(output)
    break

600
[163, 234, 50, 50, 50, 234, 50, 43, 231, 223, 151, 148, 223, 275, 149, 302, 63, 50, 70, 217, 231, 302, 302, 196, 302, 223, 196, 139, 306, 61, 223, 148, 261, 302, 139, 237, 139, 73, 196, 223, 223, 148, 223, 223, 148, 2, 231, 10, 223, 61, 50, 223, 151, 333, 61, 192, 223, 223, 350, 75, 183, 238, 32, 223, 223, 223, 302, 50, 238, 223, 87, 10, 223, 50, 32, 302, 302, 192, 139, 139, 285, 61, 75, 223, 279, 191, 10, 279, 191, 163, 139, 151, 9, 279, 173, 83, 279, 32, 285, 279, 223, 163, 279, 9, 173, 163, 61, 33, 223, 148, 247, 247, 247, 247, 247, 247, 31, 31, 61, 173, 282, 282, 247, 247, 61, 61, 282, 247, 185, 247, 31, 247, 31, 282, 282, 247, 282, 61, 61, 61, 61, 61, 61, 61, 61, 223, 223, 223, 247, 247, 223, 223, 61, 61, 61, 61, 61, 61, 61, 61, 61, 31, 247, 31, 61, 61, 61, 61, 61, 61, 61, 223, 223, 223, 223, 223, 31, 31, 31, 61, 61, 61, 223, 223, 61, 223, 247, 61, 223, 61, 223, 223, 247, 61, 61, 61, 61, 247, 223, 247, 247, 223, 223, 247, 247, 223, 247, 223, 247, 247, 223, 223, 247, 247, 223, 

/home/jakub/miniconda3/envs/midi/lib/python3.13/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


# Training

In [19]:
model = PianoTranscriber(128, len(tokenizer.id_to_token)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/home/jakub/miniconda3/envs/midi/lib/python3.13/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [20]:
def train(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    criterion: nn.CrossEntropyLoss,
    optimizer: torch.optim.Optimizer,
    epochs: int
):
    for epoch in range(epochs):
        model.train()
        for i, (image, midi) in enumerate(train_loader):
            image = image.to(device)
            midi = midi.to(device)

            input_tokens = midi[:, :-1]
            target_tokens = midi[:, 1:]
            target_tokens = target_tokens.reshape(-1)

            output = model(image, input_tokens)
            output = output.reshape(-1, output.shape[-1])

            loss: torch.Tensor = criterion(output, target_tokens)
            print(i)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

train(
    model,
    loader,
    loader,
    criterion,
    optimizer,
    epochs=1
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
